# Exceptions
### Resources
 * [Exception Heirarchy](https://docs.python.org/3/library/exceptions.html)

In [7]:
def exception_func():
    try:
        age = int(input("Enter age - "))
        x = 10 / age
    except (ValueError, ZeroDivisionError) as e:
        print("Error MSG -", e)
        print("joint exceptions!")
    except ZeroDivisionError as e:
        print("trying to divide by zero!")
    else:
        print("no exceptions thrown")
    finally:
        print("all done!")

In [8]:
exception_func()

Enter age - 1
no exceptions thrown
all done!


In [9]:
exception_func()

Enter age - 0
Error MSG - division by zero
joint exceptions!
all done!


# Copying File

In [24]:
%%writefile text.txt

Hello, its me again!
came to see you again!

Overwriting text.txt


In [25]:
with open("text.txt") as f1, open("text2.txt", "w") as f2:
    f1_content = f1.read()
    f2.write(f1_content)

In [26]:
# linux
# !cat text2.txt  

# windows
!type text2.txt  


Hello, its me again!
came to see you again!


# Calculating execution time

In [30]:
from timeit import timeit

code1 = """
def calc_factor(age):
    if age <= 0:
        raise ValueError("Age can't be zero of less!")
    return 10 / age
    
try:
    calc_factor(0)
except ValueError as e:
    #print(e)
    pass
"""

code2 = """
def calc_factor(age):
    if age <= 0:
        return None
    return 10 / age
    
if calc_factor(0) is None:
    pass
"""

print(f"code1 time - {timeit(code1, number=10000)}")
print(f"code2 time - {timeit(code2, number=10000)}")

code1 time - 0.004004599999916536
code2 time - 0.0015891000000465283


In [39]:
%%timeit -n 1000 -r 5


def calc_factor(age):
    if age <= 0:
        raise ValueError("Age can't be zero of less!")
    return 10 / age


try:
    calc_factor(0)
except ValueError as e:
    # print(e)
    pass

590 ns ± 110 ns per loop (mean ± std. dev. of 5 runs, 1000 loops each)


# Classes
 * [Magic Methods](https://rszalski.github.io/magicmethods/)

## Example 1

In [52]:
class Point:
    # class attribute, shared across all instances of class
    default_color = "red"

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Point({self.x}, {self.y})"

    # factory method can be used for complex initialization/calls
    @classmethod
    def zero(cls):
        return cls(0, 0)

    # adding 2 points
    def __add__(self, other):
        if isinstance(other, Point):
            return Point(self.x + other.x, self.y + other.y)
        raise TypeError("both should be of type Point")

    def __eq__(self, other):
        if isinstance(other, Point):
            return all([self.x == other.x, self.y == other.y])
        raise TypeError("both should be of type Point")

    def __gt__(self, other):
        if isinstance(other, Point):
            return all([self.x > other.x, self.y > other.y])
        raise TypeError("both should be of type Point")

In [53]:
print(f"{Point.default_color=}")

p1 = Point.zero()
print(f"{p1=}")
print(f"{p1.default_color=}\n")

p2 = Point(1, 2)
print(f"{p2=}")
print(f"{p2.default_color=}\n")

Point.default_color = "yellow"

print(f"{p1.default_color=}")
print(f"{p2.default_color=}\n")

p3 = Point(2, 3)
print(f"{p3.default_color=}\n")

p4 = p2 + p3
print(p4)

print(f"p4 == Point(3, 5) -> {p4 == Point(3, 5)}")
print(f"p4 > p1 -> {p4 > p1}")
print(f"p4 < p1 -> {p4 < p1}")

Point.default_color='red'
p1=Point(0, 0)
p1.default_color='red'

p2=Point(1, 2)
p2.default_color='red'

p1.default_color='yellow'
p2.default_color='yellow'

p3.default_color='yellow'

Point(3, 5)
p4 == Point(3, 5) -> True
p4 > p1 -> True
p4 < p1 -> False


## Example 2

In [65]:
class TagCloud:
    def __init__(self):
        self.tags = {}

    def add(self, tag):
        self.tags[tag.lower()] = self.tags.get(tag, 0) + 1

    def __getitem__(self, tag):
        return self.tags.get(tag, 0)

    def __setitem__(self, tag, value):
        self.tags[tag.lower()] = value

    def __len__(self):
        return len(self.tags)

    def __iter__(self):
        return iter(self.tags)

In [66]:
cloud = TagCloud()
cloud.add("Python")
cloud.add("python")
cloud.add("python")
print(cloud.tags)
print(f'{cloud["python"] = }')

cloud["abc"] = 9
print(cloud.tags)
print(f'{cloud["abc"] = }')

print(f"{len(cloud) = }")

for ele in cloud:
    print(ele)

{'python': 3}
cloud["python"] = 3
{'python': 3, 'abc': 9}
cloud["abc"] = 9
len(cloud) = 2
python
abc


## Private Members

In [67]:
class TagCloud2:
    def __init__(self):
        self.__tags = {}

    def add(self, tag):
        self.__tags[tag.lower()] = self.__tags.get(tag, 0) + 1

    def __getitem__(self, tag):
        return self.__tags.get(tag, 0)

    def __setitem__(self, tag, value):
        self.__tags[tag.lower()] = value

    def __len__(self):
        return len(self.__tags)

    def __iter__(self):
        return iter(self.__tags)

In [69]:
cloud = TagCloud2()
cloud.add("Python")
cloud.add("python")
cloud.add("python")
print(dir(cloud))

['_TagCloud2__tags', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add']


In [70]:
print(cloud._TagCloud2__tags)

{'python': 3}


## Properties

In [78]:
# UNPYTHONIC


class Product:
    def __init__(self, price):
        self.set_price(price)

    def set_price(self, price):
        if price < 0:
            raise ValueError("Price can't be negative!")
        self.__price = price

    def get_price(self):
        return self.__price

    price = property(get_price, set_price)

In [79]:
p = Product(10)
print(p.price)

p.price = 20
print(p.price)

p.price = -1
print(p.price)

10
20


ValueError: Price can't be negative!

In [83]:
class Product2:
    def __init__(self, price):
        self.price = price

    @property
    def price(self):
        return self.__price

    @price.setter
    def price(self, price):
        if price < 0:
            raise ValueError("Price can't be negative!")
        self.__price = price

In [84]:
p = Product2(10)
print(p.price)

p.price = 20
print(p.price)

p.price = -1
print(p.price)

10
20


ValueError: Price can't be negative!

## Inheritance and method overriding

In [85]:
class Animal:
    def __init__(self) -> None:
        print("animal constructor")
        self.age = 1

    def eat(self):
        print("eat")


class Mammal(Animal):
    def __init__(self) -> None:
        # base class constructor called to initialize age attribute
        # method overriding
        super().__init__()
        print("mammal constructor")
        self.weight = 2

    def walk(self):
        print("walk")


class Fish(Animal):
    def swim(self):
        print("swim")

In [87]:
m = Mammal()
m.eat()
m.walk()
print(f"{m.age = }")
print(f"{m.weight = }")
print()

f = Fish()
f.eat()
f.swim()
print()

# Useful Class Functions
print(f"{isinstance(m, Mammal) = }")
print(f"{isinstance(m, Animal) = }")
print(f"{isinstance(m, object) = }")
print(f"{issubclass(Mammal, Animal) = }")

animal constructor
mammal constructor
eat
walk
m.age = 1
m.weight = 2

animal constructor
eat
swim

isinstance(m, Mammal) = True
isinstance(m, Animal) = True
isinstance(m, object) = True
issubclass(Mammal, Animal) = True


## Abstract Base Class

In [88]:
from abc import ABC, abstractmethod


class InvalidOperationError(Exception):
    pass


class Stream(ABC):
    def __init__(self) -> None:
        self.opened = False

    def open(self):
        if self.opened:
            raise InvalidOperationError("Stream Already open.")
        self.opened = True

    def close(self):
        if not self.opened:
            raise InvalidOperationError("Stream Already closed.")
        self.opened = False

    @abstractmethod
    def read(self):
        pass


class FileStream(Stream):
    def read(self):
        print("Reading data from file.")


class NetworkStream(Stream):
    def read(self):
        print("Reading data from network.")

In [89]:
# can't directly create abstract class objects
stream = Stream()

TypeError: Can't instantiate abstract class Stream with abstract methods read

In [91]:
# need to implement abstract method or this class will also be considered abstract
class MemoryStream(Stream):
    pass


ms = MemoryStream()
ms.open()

TypeError: Can't instantiate abstract class MemoryStream with abstract methods read

In [93]:
# need to implement abstract method or this class will also be considered abstract
# now it's a concrete class
class MemoryStream(Stream):
    def read(self):
        print("reading memory stream.")


ms = MemoryStream()
ms.read()

reading memory stream.


## Polymorphism

In [94]:
class UIControl(ABC):
    @abstractmethod
    def draw(self):
        pass


class TextBox(UIControl):
    def draw(self):
        print("Textbox")


class DropDown(UIControl):
    def draw(self):
        print("DropDown")


# draw is taking different forms based on objects supplied to it
def draw(controls):
    for control in controls:
        control.draw()


ddl = DropDown()
tbox = TextBox()

draw([ddl, tbox])

DropDown
Textbox


## Extending built-in Classes

In [95]:
class TrackableList(list):
    def append(self, obj):
        print("Appending object -", obj)
        super().append(obj)


li = TrackableList()
li.append("a")
li.append("b")
li.append("c")
li.append("d")
print(li)

Appending object - a
Appending object - b
Appending object - c
Appending object - d
['a', 'b', 'c', 'd']


## Namedtuples
 * can be used to create objects which don't have any methods
 * IMMUTABLE

In [1]:
from collections import namedtuple

Point = namedtuple("Point", ["x", "y"])

p1 = Point(x=1, y=2)
p2 = Point(x=1, y=2)

print(p1 == p2)
print(p1.x, p1.y)
# immutable error
p1.x = 1

True
1 2


AttributeError: can't set attribute